In [0]:
%pip install pypdf databricks-vectorsearch
%pip install sentence-transformers
dbutils.library.restartPython()  # Restart Python to apply changes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/298.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/316.6 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.1
    Not uninstalling protobuf at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-af457319-bcdf-435b-a63a-f5e7fdcd40f5
    Can't uninstall 'protobuf'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-engineering 0.8.0 requires protobuf<5,>=3.12.0, but you have protobuf 5.28.3 which is incompatible.
tensorboard-plugin-profile 2.18.0 requires protobuf<5.0.0dev,>=3.19.6, 

In [0]:
import os
import torch
import pandas as pd
from pypdf import PdfReader
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from databricks.vector_search.client import VectorSearchClient
from transformers import AutoModel, AutoTokenizer, pipeline


2025-02-06 09:03:32.685490: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 09:03:32.693010: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 09:03:32.733572: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-06 09:03:32.770687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738832612.803573    2154 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738832612.81

In [0]:
pdf_path = "/dbfs/mnt/lake/MantisShrimpFacts.pdf" #path to your Mantis Shrimp facts PDF

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

document_text = extract_text_from_pdf(pdf_path)

print(document_text[:1000])  # Show first 1000 characters for verification


Mantis shrimp
Temporal range:
Odontodactylus scyllarus (Red Mantis
Shrimp)
Lysiosquillina maculata (Zebra Mantis
Shrimp)
Scientiﬁc classiﬁcation
Domain: Eukaryota
Kingdom: Animalia
Phylum: Arthropoda
Class: Malacostraca
Subclass: Hoplocarida
Order: Stomatopoda
Latreille, 1817
Subdivisions[ 1 ] 
Superfamilies and families
Bathysquilloidea
Bathysquillidae
Indosquillidae
Gonodactyloidea
Alainosquillidae
Hemisquillidae
Mantis shrimp
Mantis shrimp are carnivorous marine crustaceans of the
order Stomatopoda (from Ancient Greek στόµα
(stóma)  'mouth' and ποδός (podós)  'foot'). Stomatopods
branched off from other members of the class Malacostraca
around 340 million years ago.[2] Mantis shrimp typically
grow to around 10 cm (3.9 in) in length, while a few can
reach up to 38 cm (15 in).[3] A mantis shrimp's carapace
covers only the rear part of the head and the first four
segments of the thorax. Varieties range in colour from
shades of brown to vivid colours, with more than 520 species
of manti

In [0]:
def chunk_text(text, chunk_size=512):
    sentences = text.split(". ")
    chunks = []
    chunk = ""
    
    for sentence in sentences:
        if len(chunk) + len(sentence) < chunk_size:
            chunk += sentence + ". "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + ". "
    
    if chunk:
        chunks.append(chunk.strip())

    return chunks

# Apply chunking
chunks = chunk_text(document_text, chunk_size=512)

# Display chunk information
print(f"Total Chunks: {len(chunks)}")
print("Sample Chunk:\n", chunks[0])
print()
print("Sample Chunk:\n", chunks[1])
print()
print("Sample Chunk:\n", chunks[2])
print()
print("Sample Chunk:\n", chunks[-1])


Total Chunks: 113
Sample Chunk:
 

Sample Chunk:
 Mantis shrimp
Temporal range:
Odontodactylus scyllarus (Red Mantis
Shrimp)
Lysiosquillina maculata (Zebra Mantis
Shrimp)
Scientiﬁc classiﬁcation
Domain: Eukaryota
Kingdom: Animalia
Phylum: Arthropoda
Class: Malacostraca
Subclass: Hoplocarida
Order: Stomatopoda
Latreille, 1817
Subdivisions[ 1 ] 
Superfamilies and families
Bathysquilloidea
Bathysquillidae
Indosquillidae
Gonodactyloidea
Alainosquillidae
Hemisquillidae
Mantis shrimp
Mantis shrimp are carnivorous marine crustaceans of the
order Stomatopoda (from Ancient Greek στόµα
(stóma)  'mouth' and ποδός (podós)  'foot').

Sample Chunk:
 Stomatopods
branched off from other members of the class Malacostraca
around 340 million years ago.[2] Mantis shrimp typically
grow to around 10 cm (3.9 in) in length, while a few can
reach up to 38 cm (15 in).[3] A mantis shrimp's carapace
covers only the rear part of the head and the first four
segments of the thorax. Varieties range in colour from
sha

In [0]:
# Only to be uncommented if first chunk is empty. Otherwise comment it. 
chunks = chunks[1:]

In [0]:
# Convert chunks to Spark DataFrame
spark.sql("USE hive_metastore")  # Ensure we use the default catalog

df = spark.createDataFrame([(i, chunk) for i, chunk in enumerate(chunks)], ["id", "text"])

# Save to Delta Table in hive_metastore
df.write.format("delta").mode("overwrite").saveAsTable("rag_chunks")

print("Document chunks stored in Delta Table (hive_metastore.rag_chunks)")


Document chunks stored in Delta Table (hive_metastore.rag_chunks)


In [0]:
# Now let's write the data into a delta live table.
# Initialize Spark Session
spark = SparkSession.builder.appName("RAG").getOrCreate()

# Convert chunks to Spark DataFrame
df = spark.createDataFrame([(i, chunk) for i, chunk in enumerate(chunks)], ["id", "text"])

# Define Delta table path
delta_table_path = "/mnt/rag_delta_table"

# Save as Delta Table
df.write.format("delta").mode("overwrite").save(delta_table_path)
print("Text chunks stored in Delta Table.")


Text chunks stored in Delta Table.


In [0]:
from sentence_transformers import SentenceTransformer

# Load Hugging Face embedding model
embedding_model = "BAAI/bge-large-en"
model = SentenceTransformer(embedding_model)

def generate_embedding(text):
    return model.encode(text).tolist()

# Generate embeddings for each chunk
embeddings = [(i, generate_embedding(chunk)) for i, chunk in enumerate(chunks)]

# Convert to Spark DataFrame
embeddings_df = spark.createDataFrame(embeddings, ["id", "embedding"])

# Save embeddings to Delta Table in hive_metastore
embeddings_df.write.format("delta").mode("overwrite").saveAsTable("rag_embeddings")

print("Embeddings stored in Delta Table (hive_metastore.rag_embeddings)")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Embeddings stored in Delta Table (hive_metastore.rag_embeddings)


In [0]:
from pyspark.sql.functions import col
import numpy as np

# Function to compute cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Define a query embedding
query_text = "What does mantis shrimp eat?"
query_embedding = model.encode(query_text).tolist()

# Load embeddings from Delta Table
df_embeddings = spark.sql("SELECT * FROM rag_embeddings").toPandas()

# Compute similarity scores
df_embeddings["similarity"] = df_embeddings["embedding"].apply(lambda x: cosine_similarity(query_embedding, x))

# Get top 3 most relevant chunks
top_chunks = df_embeddings.sort_values(by="similarity", ascending=False).head(3)

# Retrieve corresponding text chunks
retrieved_chunks = spark.sql("SELECT * FROM rag_chunks").toPandas()
retrieved_texts = retrieved_chunks[retrieved_chunks["id"].isin(top_chunks["id"])]["text"].tolist()

print("🔎 Retrieved Chunks:")
for text in retrieved_texts:
    print('******')
    print(text)


🔎 Retrieved Chunks:
******
has been observed to roll repeatedly for 2 m (6.6 ft), but specimens typically travel
less than 1 m (3.3 ft).[51]
The mantis shrimp is eaten by a variety of cultures. In Japanese
cuisine, the mantis shrimp species Oratosquilla oratoria,
called shako ( 蝦蛄), is eaten boiled as a sushi topping, and
occasionally raw as sashimi.
Mantis shrimps are also abundant along Vietnam's coast,
known in Vietnamese as bề bề, tôm tích or tôm tít.
******
Additionally,
some rock-burrowing species can do more damage to live rock
than the fishkeeper would prefer.
The live rock with mantis shrimp burrows is considered useful
by some in the marine aquarium trade and is often collected. A
piece of live rock not uncommonly conveys a live mantis
shrimp into an aquarium.
******
However, despite being
common, they are poorly understood, as many species spend
most of their lives sheltering in burrows and holes.[4]
Called "sea locusts" by ancient Assyrians, "prawn killers" in
Australia,[5]

In [0]:
from transformers import pipeline

# Load Hugging Face LLM 
llm_pipeline = pipeline("text-generation", model="facebook/opt-1.3b")
# very small model-> facebook/opt-6.7b is better but may crash the kernel 

# Generate response based on retrieved context
context = "\n".join(retrieved_texts)
prompt = f"Context: {context}\n\nQuestion: {query_text}\nAnswer:"

response = llm_pipeline(prompt, max_length=150, num_return_sequences=1)
print("💡 AI Response:\n", response[0]["generated_text"])


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


💡 AI Response:
 Context: has been observed to roll repeatedly for 2 m (6.6 ft), but specimens typically travel
less than 1 m (3.3 ft).[51]
The mantis shrimp is eaten by a variety of cultures. In Japanese
cuisine, the mantis shrimp species Oratosquilla oratoria,
called shako ( 蝦蛄), is eaten boiled as a sushi topping, and
occasionally raw as sashimi.
Mantis shrimps are also abundant along Vietnam's coast,
known in Vietnamese as bề bề, tôm tích or tôm tít.
Additionally,
some rock-burrowing species can do more damage to live rock
than the fishkeeper would prefer.
The live rock with mantis shrimp burrows is considered useful
by some in the marine aquarium trade and is often collected. A
piece of live rock not uncommonly conveys a live mantis
shrimp into an aquarium.
However, despite being
common, they are poorly understood, as many species spend
most of their lives sheltering in burrows and holes.[4]
Called "sea locusts" by ancient Assyrians, "prawn killers" in
Australia,[5] and now sometim

Below code is for high level understanding and WILL NOT EXECUTE. 

In [0]:
# WARNING: FOLLOWING CODE WILL NOT WORK AND THROW ERROR DUE TO UNITY CATALOG/PERMISSIONS 

# Part 1- building embeddings and vectors

import mlflow
from databricks.vector_search.client import VectorSearchClient
from pyspark.sql.functions import col

# Initialize Databricks Vector Search Client
vsc = VectorSearchClient()

# Define Unity Catalog schema and table
catalog_name = "your_catalog"
schema_name = "your_schema"
vector_index_name = "your_vector_index"

# Load Databricks Model for Embeddings
embedding_model_uri = "models:/your_embedding_model/production"
embedding_model = mlflow.pyfunc.load_model(embedding_model_uri)

def generate_embeddings(texts):
    """Generate embeddings using Databricks MLflow Model."""
    return [embedding_model.predict([text])[0] for text in texts]

# Create or load the Vector Search index
try:
    vsc.get_index(catalog_name, schema_name, vector_index_name)
except Exception:
    vsc.create_index(catalog_name, schema_name, vector_index_name, dimension=768)

# Load dataset from Unity Catalog
df = spark.read.table(f"{catalog_name}.{schema_name}.your_text_table")

# Generate embeddings and store them in Vector Search
text_data = df.select("text_column").rdd.map(lambda row: row[0]).collect()
embeddings = generate_embeddings(text_data)

# Convert to DataFrame and store in Vector Search
vector_df = spark.createDataFrame(
    [(i, text, embedding) for i, (text, embedding) in enumerate(zip(text_data, embeddings))],
    ["id", "text", "embedding"]
)

vector_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{vector_index_name}")

print("Embeddings stored in Unity Catalog and Databricks Vector Search.")


/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:60: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:2570: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:3431: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


/databricks/python/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


---------------------------------------------------------------------------
RestException                             Traceback (most recent call last)
File <command-7838288942542868>, line 19
     17 # Load Databricks Model for Embeddings
     18 embedding_model_uri = "models:/your_embedding_model/production"
---> 19 embedding_model = mlflow.pyfunc.load_model(embedding_model_uri)
     21 def generate_embeddings(texts):
     22     """Generate embeddings using Databricks MLflow Model."""

File /databricks/python/lib/python3.12/site-packages/mlflow/tracing/provider.py:237, in trace_disabled.<locals>.wrapper(*args, **kwargs)
    235 disable()
    236 try:
--> 237     is_func_called, result = True, f(*args, **kwargs)
    238 finally:
    239     enable()

File /databricks/python/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py:993, in load_model(model_uri, suppress_warnings, dst_path, model_config)
    989         entity_list.append(Entity(job=job_entity))
    991     lineage_header

In [0]:
# WARNING: FOLLOWING CODE WILL NOT WORK AND THROW ERROR DUE TO UNITY CATALOG/PERMISSIONS

# Part 2- RAG 

# Load a Databricks-hosted LLM model for RAG
llm_model_uri = "models:/your_llm_model/production"
llm_model = mlflow.pyfunc.load_model(llm_model_uri)

def retrieve_relevant_chunks(query, top_k=5):
    """Retrieve relevant text chunks using Databricks Vector Search."""
    query_embedding = embedding_model.predict([query])[0]
    results = vsc.search(
        catalog_name, schema_name, vector_index_name,
        query_embedding, top_k=top_k
    )
    return [row["text"] for row in results]

def generate_response(query):
    """Generate response using retrieved chunks and LLM."""
    relevant_chunks = retrieve_relevant_chunks(query)
    context = "\n".join(relevant_chunks)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    response = llm_model.predict([prompt])[0]
    return response

# Example usage
query = "What is a Mantis Shrimp?"
response = generate_response(query)
print("Generated Response:", response)
